In [86]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
from pathlib import Path

In [87]:
tickers_path = "Resources/tickers_list/tickers_list.csv"

tickers_list = pd.read_csv(tickers_path).to_dict()
tickers_list['tickers'].values()

for ticker in tickers_list['tickers'].values():
    csvpath = f"Resources/{ticker}.csv"
    print(csvpath)

Resources/ENPH.csv
Resources/SEDG.csv
Resources/CMG.csv
Resources/VRTX.csv
Resources/BMY.csv
Resources/WRB.csv


In [88]:
for ticker in tickers_list['tickers'].values():
    csvpath = f"Resources/{ticker}.csv"
    df = pd.read_csv(csvpath, index_col=0, infer_datetime_format=True, parse_dates=True).dropna()
    tickers_name = f"{ticker} \nTechnical Indicators"

    # Create variables for each plot
    sma50_plot = df['SMA 50'].hvplot.line()
    sma200_plot = df['SMA 200'].hvplot.line()
    ema20_plot = df['EMA 20'].hvplot.line()
    sma20_plot = df['SMA 20'].hvplot.line()
    rsi_plot = df['RSI'].hvplot.line(height = 150)

    # Overlay Plots
    final_plot = (
        (sma200_plot * sma50_plot * ema20_plot * sma20_plot).opts(title=tickers_name, height=500).opts(legend_position='bottom_right') 
        +
        (rsi_plot * hv.HLine(50).opts(color='red', line_dash='dashed'))
    ).cols(1)
    display(final_plot)

:Layout
   .Overlay.I   :Overlay
      .Curve.SMA_200 :Curve   [index]   (SMA 200)
      .Curve.SMA_50  :Curve   [index]   (SMA 50)
      .Curve.EMA_20  :Curve   [index]   (EMA 20)
      .Curve.SMA_20  :Curve   [index]   (SMA 20)
   .Overlay.RSI :Overlay
      .Curve.RSI :Curve   [index]   (RSI)
      .HLine.I   :HLine   [x,y]

:Layout
   .Overlay.I   :Overlay
      .Curve.SMA_200 :Curve   [index]   (SMA 200)
      .Curve.SMA_50  :Curve   [index]   (SMA 50)
      .Curve.EMA_20  :Curve   [index]   (EMA 20)
      .Curve.SMA_20  :Curve   [index]   (SMA 20)
   .Overlay.RSI :Overlay
      .Curve.RSI :Curve   [index]   (RSI)
      .HLine.I   :HLine   [x,y]

:Layout
   .Overlay.I   :Overlay
      .Curve.SMA_200 :Curve   [index]   (SMA 200)
      .Curve.SMA_50  :Curve   [index]   (SMA 50)
      .Curve.EMA_20  :Curve   [index]   (EMA 20)
      .Curve.SMA_20  :Curve   [index]   (SMA 20)
   .Overlay.RSI :Overlay
      .Curve.RSI :Curve   [index]   (RSI)
      .HLine.I   :HLine   [x,y]

:Layout
   .Overlay.I   :Overlay
      .Curve.SMA_200 :Curve   [index]   (SMA 200)
      .Curve.SMA_50  :Curve   [index]   (SMA 50)
      .Curve.EMA_20  :Curve   [index]   (EMA 20)
      .Curve.SMA_20  :Curve   [index]   (SMA 20)
   .Overlay.RSI :Overlay
      .Curve.RSI :Curve   [index]   (RSI)
      .HLine.I   :HLine   [x,y]

:Layout
   .Overlay.I   :Overlay
      .Curve.SMA_200 :Curve   [index]   (SMA 200)
      .Curve.SMA_50  :Curve   [index]   (SMA 50)
      .Curve.EMA_20  :Curve   [index]   (EMA 20)
      .Curve.SMA_20  :Curve   [index]   (SMA 20)
   .Overlay.RSI :Overlay
      .Curve.RSI :Curve   [index]   (RSI)
      .HLine.I   :HLine   [x,y]

:Layout
   .Overlay.I   :Overlay
      .Curve.SMA_200 :Curve   [index]   (SMA 200)
      .Curve.SMA_50  :Curve   [index]   (SMA 50)
      .Curve.EMA_20  :Curve   [index]   (EMA 20)
      .Curve.SMA_20  :Curve   [index]   (SMA 20)
   .Overlay.RSI :Overlay
      .Curve.RSI :Curve   [index]   (RSI)
      .HLine.I   :HLine   [x,y]